<a href="https://colab.research.google.com/github/dhanushnayak/BERT/blob/main/CoLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/in_domain_train.tsv",delimiter='\t',header=None,names=["sen","label","label_notes","sentence"])

In [2]:
df.shape

(8551, 4)

In [3]:
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-chwkqwoo
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-chwkqwoo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
!pip list | grep -E "transformers|tokenizers"

tokenizers                    0.12.1
transformers                  4.21.0.dev0


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification,BertTokenizer,BertConfig

In [6]:
text = df.sentence.to_list()
#sent = ["[CLS] "+x+" [SEP]" for x in sent]
labels = df.label.to_list()

In [7]:
x_train,x_test,y_train,y_test = train_test_split(text,labels,test_size=0.2)

In [8]:
from torch.utils.data import TensorDataset,Dataset
class CreateDataset(Dataset):
  def __init__(self,encoding,labels):
    self.encoding = encoding
    self.labels = labels
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encoding.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

tokenize the sent 

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased',do_lower_case=True)


In [10]:
train_encoding = tokenizer(x_train,truncation=True,padding=True)
test_encoding = tokenizer(x_test,truncation=True,padding=True)

In [36]:
train_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [11]:
train_dataset = CreateDataset(train_encoding,y_train)
test_dataset = CreateDataset(test_encoding,y_test)

MODEL

In [12]:
from transformers import BertForSequenceClassification,BertForTokenClassification,BertTokenizer

In [13]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
from transformers import TrainingArguments,Trainer
arg = TrainingArguments(
    output_dir = "/output_result",
    num_train_epochs = 1,
    logging_dir = '/logs',
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size = 64,
    weight_decay=0.01,
    learning_rate=5e-5
)

In [16]:
trainer = Trainer(
    model=model,
    args=arg,
    train_dataset =  train_dataset,
    eval_dataset = test_dataset
)

In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6840
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 428


Step,Training Loss
10,0.639000
20,0.598200
30,0.628000
40,0.621100
50,0.579600
60,0.604500
70,0.512300
80,0.586400
90,0.579600
100,0.502900




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=428, training_loss=0.5034741848428673, metrics={'train_runtime': 63.3939, 'train_samples_per_second': 107.897, 'train_steps_per_second': 6.751, 'total_flos': 140599970208000.0, 'train_loss': 0.5034741848428673, 'epoch': 1.0})

Native Pytorch

In [19]:
from torch.utils.data import Dataset
from transformers import AdamW

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [20]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from c

In [21]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)

In [25]:
optim = AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [31]:
num_epoch=3
for epoc in range(num_epoch):
  for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    out = model(input_ids,attention_mask=attention_mask,labels=labels)
    """print(out)
    print(len(out),"len")
    break"""
    loss = out[0]
    loss.backward()
    optim.step()


In [32]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element